In [17]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import tensorflow as tf

from models import vgg
from utils.angles import deg2bit, bit2deg
from utils.losses import maad_from_deg 
from utils.losses import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

In [57]:
np.exp(-1.073867)

0.34168466480059195

In [58]:
np.exp( -1.037539)

0.35432560516251921

In [2]:
xtr, ytr_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yte_bit = deg2bit(yte_deg)

In [59]:
X = Input(shape=[50,50,3])

vgg_x = vgg.vgg_model(final_layer=False,
                      image_height=50,
                      image_width=50)(X)

mu = Lambda(lambda x: K.l2_normalize(x, axis=1))(Dense(2)(vgg_x))

kappa_pred = tf.ones([tf.shape(mu)[0],1])

# kappa_pred = Lambda(lambda x: K.abs(x))(Dense(1)(vgg_x))

def _von_mises_neg_log_likelihood_keras(y_true, y_pred):
    return -von_mises_log_likelihood_tf(y_true, y_pred, kappa_pred, input_type='biternion')

biternion_vgg_prob = Model(X, mu)
# kappa_model = Model(X, concatenate([kappa_pred, kappa_pred]))

optimizer = keras.optimizers.Adam(lr=0.0001)

biternion_vgg_prob.compile(loss=_von_mises_neg_log_likelihood_keras,
                           metrics=['cosine'],
                           optimizer=optimizer)

In [ ]:
biternion_vgg_prob.fit(x=xtr , y=ytr_bit, batch_size=10, validation_split=0.1, epochs=20)

In [22]:
yte_preds_bit = biternion_vgg_prob.predict(xte)
yte_preds_deg = deg2bit(yte_preds_bit)

In [36]:
von_mises_log_likelihood_np(yte_bit, yte_bit, np.ones([yte_bit.shape[0],1])*200, input_type='biternion')

1.7295935793671882

In [10]:
kappa_model = Model(X, kappa_pred)

In [16]:
kappa_te_preds = kappa_model.predict(xte)

In [13]:
loss = maad_from_deg(yte_preds, yte_deg)

mean_loss = np.mean(loss)
std_loss = np.std(loss)

print("MAAD error (test) : %f ± %f" % (mean_loss, std_loss))

MAAD error (test) : 79.814962 ± 53.066745
